In [1]:
import json, requests, random, os

In [ ]:
! wget -O res.json 'https://esgf-node.llnl.gov/esg-search/search?type=Dataset&limit=1000&from=NOW-3YEAR&format=application%2fsolr%2bjson&replica=true&data_node=esgf-data1.llnl.gov&retracted=false&latest=false&fields=master_id'

In [2]:
obj = json.load(open('res.json'))

In [3]:
lst = list(range(1000))
random.shuffle(lst)
master_id = obj["response"]["docs"][lst[0]]["master_id"]
relpath = master_id.replace('.','/')
abspath = f"/p/css03/esgf_publish/{relpath}"
print(abspath)

/p/css03/esgf_publish/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3-Veg/ssp245/r5i1p1f1/Emon/fNup/gr


In [4]:
dirlst = list(os.listdir(abspath))
dirlst.sort()
for dn in dirlst:    
    print(f"{master_id}.{dn}.txt   {abspath}/{dn}")


CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.r5i1p1f1.Emon.fNup.gr.v20200312.txt   /p/css03/esgf_publish/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3-Veg/ssp245/r5i1p1f1/Emon/fNup/gr/v20200312
CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.r5i1p1f1.Emon.fNup.gr.v20210616.txt   /p/css03/esgf_publish/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3-Veg/ssp245/r5i1p1f1/Emon/fNup/gr/v20210616


In [5]:
METAFILE="CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.r5i1p1f1.Emon.fNup.gr.v20200312.txt"

In [6]:
%%sh 

outfile="CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.r5i1p1f1.Emon.fNup.gr.v20200312.txt"
#wget https://raw.githubusercontent.com/sashakames/aims-pub-util/master/get_meta.sh
dn="/p/css03/esgf_publish/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3-Veg/ssp245/r5i1p1f1/Emon/fNup/gr/v20200312"  # project should match identifier above 
for fn in `find $dn -name "*.nc"` ; do 
  bash ../aims-pub-util/get_meta.sh $fn ; done > $outfile  #  file name should match const above


In [7]:
arr = []
with open(METAFILE) as f:
    for line in f:
        arr.append(line.rstrip())

In [8]:
FIRST_ID="CMIP6"
out_dict = {}

for line in arr:

    parts = line.split()
    path = parts[1]

    checksum = parts[0]

    ts = parts[2]  # timestamp

    sz = parts[3].strip()  # size

    pp2 = path.split('/')
    idx = pp2.index(FIRST_ID)
    last = len(pp2) - 2
    dset_id = ".".join(pp2[idx  :last]) + "#" + pp2[last][1:]

    out_arr = []
    out_arr.append(dset_id)
    out_arr.append(path)
    out_arr.append(sz)
    out_arr.append("mod_time=" + ts)
    out_arr.append("checksum=" + checksum)
    out_arr.append("checksum_type=SHA256")
    out_line = ' | '.join(out_arr) + "\n"
    
    if not dset_id in out_dict:
        out_dict[dset_id] = [out_line]
    else:
        out_dict[dset_id].append(out_line)

for dset_id in out_dict:
    outfilename = dset_id.replace("#",".v") + ".map"
    print("Writing " + outfilename)
    with open(outfilename , "w") as f:
        for line in out_dict[dset_id]:
            f.write(line)

Writing CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.r5i1p1f1.Emon.fNup.gr.v20200312.map
